In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)
wiki.name

'wikipedia'

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings())
retriver = vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F7587B55D0>)

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [6]:
retriver_tool.name

'langsmith_search'

In [7]:
# Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [21]:
arxiv.name

'arxiv'

In [8]:
tools = [wiki, retriver_tool, arxiv]

In [ ]:
tool_names = ['wikipedia', 'langsmith_search', 'arxiv']

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model = 'llama2')
llm

Ollama()

In [14]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("hwchase17/react")


In [20]:
from langchain.agents import create_react_agent

agent = create_react_agent(llm, tools, prompt)

ValueError: Prompt missing required variables: {'tools', 'agent_scratchpad', 'tool_names'}

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor.invoke({"input": "Tell me about LangSmith"})




> Entering new AgentExecutor chain...
Action: langsmith_search(query: 'LangSmith', retriever: 'BaseRetriever', document_prompt: 'BasePromptTemplate', document_separator: '\n\n', callbacks: 'Callbacks')
Action Input: LangSmith querylangsmith_search(query: 'LangSmith', retriever: 'BaseRetriever', document_prompt: 'BasePromptTemplate', document_separator: '\n\n', callbacks: 'Callbacks') is not a valid tool, try one of [wikipedia, langsmith_search, arxiv].Question: Tell me about LangSmith
Thought: To find information about LangSmith, I will use the `langsmith_search` tool.
Action: langsmith_search(query: 'LangSmith', retriever: 'BaseRetriever', document_prompt: 'BasePromptTemplate', document_separator: '\n\n', callbacks: 'Callbacks')
Action Input: LangSmith querylangsmith_search(query: 'LangSmith', retriever: 'BaseRetriever', document_prompt: 'BasePromptTemplate', document_separator: '\n\n', callbacks: 'Callbacks') is not a valid tool, try one of [wikipedia, langsmith_search, arxiv].Ques

KeyboardInterrupt: 